# Database interaction

In this section, I want to mention details about interfacing with databases from Pandas.

## SQLAlchemy

There is a very important library for organising the interaction between the database and the pandas - <a href="https://www.sqlalchemy.org/">SQLAlchemy</a>.

<a href="https://docs.sqlalchemy.org/en/20/core/engines.html#postgresql">Create engine in SQlAlchemy</a>.

# Postgres

## Read information

You can use `pandas.read_sql` and pass the created SQLAlchemy engine as `con` argument.

So in the following example, I start the postgres database in the docker container and then read information from it directly to pandas.

[SQL script](postgres_inter_files/simple_example_db/create_table.sql) for initialisation postgres datase used in this example.

In [3]:
!docker run --rm -d\
    -v ./postgres_inter_files/simple_example_db:/docker-entrypoint-initdb.d\
    -e POSTGRES_PASSWORD=docker_app\
    --name read_postgres\
    -p 5431:5432\
    postgres:15.4 &> /dev/null
!sleep 10

import pandas as pd
from sqlalchemy import URL, create_engine
url_object = URL.create(
    "postgresql+psycopg2",
    username="postgres",
    password="docker_app",
    host="localhost",
    port=5431,
    database="postgres",
)
engine = create_engine(url_object)

df = pd.read_sql('SELECT * FROM main_table', con=engine, index_col = "id")
print(df)
!docker stop read_postgres &> /dev/null

     text
id       
0   Text1
1   tExT2
3   TEXT3


# No SQLAlchemy

Actually, it's possible to use the Postgres database without using SQLAlchemy - just pass the `connection object` from `psycopg2` to the `con` parameter of the `read_sql` function.

In the following example, that's exactly what I did - but I got a warning that the pandas connection was only tested with sqlAlchemy, so it's better to use it.

In [18]:
!docker run --rm -d\
    -v ./postgres_inter_files/simple_example_db:/docker-entrypoint-initdb.d\
    -e POSTGRES_PASSWORD=docker_app \
    --name read_info_example \
    -p 5431:5432\
    postgres:15.4 &> /dev/null
!sleep 1

import pandas as pd
import psycopg2

conn = psycopg2.connect(
    port = "5431", # same as when creating a postgres container
    dbname = "postgres",
    user = "postgres",
    password = "docker_app",
    host= "localhost"
)

df = pd.read_sql('SELECT * FROM main_table', con=conn)
print(df)
conn.close()

!docker stop read_info_example &> /dev/null

  id   text
0  0  Text1
1  1  tExT2
2  3  TEXT3


/tmp/ipykernel_5907/3499609097.py:15: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql('SELECT * FROM main_table', con=conn)
